# Importing Modules and Defining Functions

In [1]:
import re
import sys
import requests
import pandas as pd
from json import loads

def httpsReq(requestAddr):

    #Making the API Request
    # print(f'Making API Call to: {requestAddr}')
    response = requests.get(requestAddr)
    if response.status_code == 200:
        pass
    elif response.status_code != 200:
        print(f'{response.status_code} - API Request {requestAddr} Failed')
        KeyboardInterrupt

    #Converting JSON to Dict and Parsing API Data into a Dataframe
    dict = loads(response.text)

    return dict

# Requesting Game History via Chess.com API
### We need to stage our api request to transform into a dataframe:
- The api request address only returns the games for the selected month and year
- There we must pull down our archive histotry and loop through the api requests to return data for each month

In [2]:
#Retrieveing Archive of My Entire Chess.com History
dict = httpsReq('https://api.chess.com/pub/player/filiplivancic/games/archives')
print(f'API Request for Monthly API Endpoints Successful')
archive = list(dict['archives'])
archive[:5]

API Request https://api.chess.com/pub/player/filiplivancic/games/archives Successful


['https://api.chess.com/pub/player/filiplivancic/games/2019/05',
 'https://api.chess.com/pub/player/filiplivancic/games/2019/07',
 'https://api.chess.com/pub/player/filiplivancic/games/2019/08',
 'https://api.chess.com/pub/player/filiplivancic/games/2019/09',
 'https://api.chess.com/pub/player/filiplivancic/games/2019/10']

## Looping through every archived month, pulling all games and placing into list

In [3]:
list = []
print('Extracting Entire Game History Through All Monthly API Endpoints')
for request in archive:
    dict_arch = httpsReq(request)
    list.append(dict_arch)
print(f'API Request Successful and Appended all the dictionaries together')

API Request https://api.chess.com/pub/player/filiplivancic/games/2019/05 Successful
API Request https://api.chess.com/pub/player/filiplivancic/games/2019/07 Successful
API Request https://api.chess.com/pub/player/filiplivancic/games/2019/08 Successful
API Request https://api.chess.com/pub/player/filiplivancic/games/2019/09 Successful
API Request https://api.chess.com/pub/player/filiplivancic/games/2019/10 Successful
API Request https://api.chess.com/pub/player/filiplivancic/games/2019/11 Successful
API Request https://api.chess.com/pub/player/filiplivancic/games/2019/12 Successful
API Request https://api.chess.com/pub/player/filiplivancic/games/2020/03 Successful
API Request https://api.chess.com/pub/player/filiplivancic/games/2020/04 Successful
API Request https://api.chess.com/pub/player/filiplivancic/games/2020/05 Successful
API Request https://api.chess.com/pub/player/filiplivancic/games/2020/06 Successful
API Request https://api.chess.com/pub/player/filiplivancic/games/2020/07 Suc

## Unpacking List of Games

In [4]:
game_list = []
print('Transforming JSON data into a dataframe')
for month in list: 
    month_of_games = month['games']
    for game in month_of_games: game_list.append(game)
df = pd.DataFrame(game_list)
df.head(3)

,url,pgn,time_control,end_time,rated,tcn,uuid,initial_setup,fen,time_class,rules,white,black,accuracies
0,https://www.chess.com/game/live/3668045802,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1556902183,True,mC3Nbs5Qlt!TcMZJCKQKtB7RfH6Zdm86BKRzKT0THOzjOX...,fd899364-2d30-11e5-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,7r/pkp2pp1/8/3p3p/4r3/8/P1P1KPPP/q3b3 w - -,blitz,chess,"{'rating': 804, 'result': 'timeout', '@id': 'h...","{'rating': 973, 'result': 'win', '@id': 'https...",NaN
1,https://www.chess.com/game/live/3668084540,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1556902975,True,lB!Tbs5QBJQKnDKEpxEVmC0SCKT!sC9zclWGkA7FowF0iq...,a090dbf8-2d36-11e5-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,2k1r2r/1pp4p/3p1p1n/P2P1n2/4K1B1/6q1/1P1B4/R2Q...,blitz,chess,"{'rating': 797, 'result': 'checkmated', '@id':...","{'rating': 925, 'result': 'win', '@id': 'https...",NaN
2,https://www.chess.com/game/live/3668146117,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1556903918,True,ltWGcM2Ugv4OmCYItBIAfAOPbsPQBJQAvKAYdvZRv1,96bfa632-2d3f-11e5-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,1nbqkbnr/1pr1pQ1p/3p2p1/p2PN1B1/4P3/2N5/PPP2PP...,blitz,chess,"{'rating': 990, 'result': 'win', '@id': 'https...","{'rating': 775, 'result': 'checkmated', '@id':...",NaN


## Unpacking Nested white/black dictionaries -- Creating Derived Data For Wins/Losses

In [5]:
# Checking contents of white/black
print('Unpacking the ratings data from all individual games')
result_list = []
for i in range(df.shape[0]):

    #Find each dictionary set for white and black
    white_result = df.iloc[i,df.columns.get_loc('white')]
    black_result = df.iloc[i,df.columns.get_loc('black')]

    #Search for my username, if there is a match, we will append it to the results list
    if white_result['username'] == 'filiplivancic':
        result_list.append(white_result)
    else:
        result_list.append(black_result)
    
#Performing check to see if length of list matches the size of the dataframe
print(df.shape[0],len(result_list))

#Transforming results into output dataframe and dropping irrelevant columns
result_df = pd.DataFrame(result_list)
result_df.drop(columns={'@id','username','uuid'}, inplace=True)

#Adding extra col to group the varieties of losses and draws into a parent catagory
result_df['win/loss'] = ''
for i in range(result_df.shape[0]):
    res = result_df.iloc[i,result_df.columns.get_loc('result')]
    if res == 'win':
        result_df.iloc[i,result_df.columns.get_loc('win/loss')] = 'win'
    if res == 'timevsinsufficient' or res == 'stalemate':
        result_df.iloc[i,result_df.columns.get_loc('win/loss')] = 'draw'
    if res != 'win' and res != 'timevsinsufficient' and res != 'stalemate':
        result_df.iloc[i,result_df.columns.get_loc('win/loss')] = 'loss'
# set(result_df['result'])
# result_df.loc[result_df['win/loss'] == 'draw']
result_df


2242 2242


,rating,result,win/loss
0,804,timeout,loss
1,925,win,win
2,990,win,win
3,897,resigned,loss
4,828,resigned,loss
...,...,...,...
2237,1023,win,win
2238,1012,resigned,loss
2239,1021,win,win
2240,1031,win,win


## Concatenating Result Onto The Main Dataframe

In [6]:
print('Appending the unpacked data onto the main dataframe')
df_rating = pd.concat([df,result_df],axis=1)
df_rating.head(3)

,url,pgn,time_control,end_time,rated,tcn,uuid,initial_setup,fen,time_class,rules,white,black,accuracies,rating,result,win/loss
0,https://www.chess.com/game/live/3668045802,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1556902183,True,mC3Nbs5Qlt!TcMZJCKQKtB7RfH6Zdm86BKRzKT0THOzjOX...,fd899364-2d30-11e5-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,7r/pkp2pp1/8/3p3p/4r3/8/P1P1KPPP/q3b3 w - -,blitz,chess,"{'rating': 804, 'result': 'timeout', '@id': 'h...","{'rating': 973, 'result': 'win', '@id': 'https...",NaN,804,timeout,loss
1,https://www.chess.com/game/live/3668084540,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1556902975,True,lB!Tbs5QBJQKnDKEpxEVmC0SCKT!sC9zclWGkA7FowF0iq...,a090dbf8-2d36-11e5-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,2k1r2r/1pp4p/3p1p1n/P2P1n2/4K1B1/6q1/1P1B4/R2Q...,blitz,chess,"{'rating': 797, 'result': 'checkmated', '@id':...","{'rating': 925, 'result': 'win', '@id': 'https...",NaN,925,win,win
2,https://www.chess.com/game/live/3668146117,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1556903918,True,ltWGcM2Ugv4OmCYItBIAfAOPbsPQBJQAvKAYdvZRv1,96bfa632-2d3f-11e5-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,1nbqkbnr/1pr1pQ1p/3p2p1/p2PN1B1/4P3/2N5/PPP2PP...,blitz,chess,"{'rating': 990, 'result': 'win', '@id': 'https...","{'rating': 775, 'result': 'checkmated', '@id':...",NaN,990,win,win


## Extracting More Metadata From The Chess Games

In [7]:
date_list = []
opening_list = []
quoted = re.compile('"[^"]*"')

for i in range(df_rating.shape[0]):
    
    #Splitting the metadata by each new line
    meta_data = df.iloc[i,df.columns.get_loc('pgn')].splitlines(False)
    
    #Appending dates into the list by string index 
    # date_list.append(meta_data[2][7:17])
    for date in quoted.findall(meta_data[2]): date_list.append(date)

    #Appending openings by retrieveing double quote contents
    for opening in quoted.findall(meta_data[10]): opening_list.append(opening)


## Adding Metadata as New Columns in the DataFrame

In [8]:
df['Date'] = date_list
df['Opening'] = opening_list
df.head(5)

,url,pgn,time_control,end_time,rated,tcn,uuid,initial_setup,fen,time_class,rules,white,black,accuracies,Date,Opening
0,https://www.chess.com/game/live/3668045802,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1556902183,True,mC3Nbs5Qlt!TcMZJCKQKtB7RfH6Zdm86BKRzKT0THOzjOX...,fd899364-2d30-11e5-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,7r/pkp2pp1/8/3p3p/4r3/8/P1P1KPPP/q3b3 w - -,blitz,chess,"{'rating': 804, 'result': 'timeout', '@id': 'h...","{'rating': 973, 'result': 'win', '@id': 'https...",NaN,"""2019.05.03""","""https://www.chess.com/openings/Kings-Pawn-Ope..."
1,https://www.chess.com/game/live/3668084540,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1556902975,True,lB!Tbs5QBJQKnDKEpxEVmC0SCKT!sC9zclWGkA7FowF0iq...,a090dbf8-2d36-11e5-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,2k1r2r/1pp4p/3p1p1n/P2P1n2/4K1B1/6q1/1P1B4/R2Q...,blitz,chess,"{'rating': 797, 'result': 'checkmated', '@id':...","{'rating': 925, 'result': 'win', '@id': 'https...",NaN,"""2019.05.03""","""https://www.chess.com/openings/Indian-Game-2...."
2,https://www.chess.com/game/live/3668146117,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1556903918,True,ltWGcM2Ugv4OmCYItBIAfAOPbsPQBJQAvKAYdvZRv1,96bfa632-2d3f-11e5-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,1nbqkbnr/1pr1pQ1p/3p2p1/p2PN1B1/4P3/2N5/PPP2PP...,blitz,chess,"{'rating': 990, 'result': 'win', '@id': 'https...","{'rating': 775, 'result': 'checkmated', '@id':...",NaN,"""2019.05.03""","""https://www.chess.com/openings/Mieses-Opening"""
3,https://www.chess.com/game/live/3668401579,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1556910410,True,mC2UfA!Tlt92gv5QbsZRvMQzM17Z1?2?iqZyAryGeg6Env...,c339e96e-2d64-11e5-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,r3k2b/ppp1p2p/1q1p1np1/8/4P1b1/PBNP1P2/1PQ3PP/...,blitz,chess,"{'rating': 930, 'result': 'win', '@id': 'https...","{'rating': 897, 'result': 'resigned', '@id': '...",NaN,"""2019.05.03""","""https://www.chess.com/openings/Modern-Defense..."
4,https://www.chess.com/game/live/3668434614,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1556911325,True,mCYQnv2UlB3Now!Tks5OjzZRfOXOiy45yGNFcMFwMT0Tdl...,920fe7bc-2d69-11e5-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,1r3r2/p4p2/p1pp1kp1/P7/1P1PP3/2P3PR/3Nb2Q/R3K3...,blitz,chess,"{'rating': 908, 'result': 'win', '@id': 'https...","{'rating': 828, 'result': 'resigned', '@id': '...",NaN,"""2019.05.03""","""https://www.chess.com/openings/Caro-Kann-Defe..."
